In [23]:
!pip install opencv-python ipywidgets

In [30]:
import cv2
import numpy as np
from IPython.display import display, clear_output
from ipywidgets import widgets
from IPython.display import Image
import threading

# Load pretrained model for gender detection (using Caffe model here)
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Pretrained models: deploy architecture and gender model weights
gender_net = cv2.dnn.readNetFromCaffe(
    'gender_deploy.prototxt',  # Path to your .prototxt file
    'gender_net.caffemodel'    # Path to your .caffemodel file
)

# Label mappings for gender prediction
GENDER_LIST = ['Male', 'Female']

# Load a pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Global variables to control the webcam
cap = None
is_running = False

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = GENDER_LIST[gender_preds[0].argmax()]
        color = (0, 0, 255) if gender == 'Male' else (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, gender, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    return frame

def start_webcam():
    global cap, is_running
    if cap is None:
        cap = cv2.VideoCapture(0)
    while is_running:
        ret, frame = cap.read()
        if not ret:
            break
        # Flip the frame horizontally to correct inversion
        frame = cv2.flip(frame, 1)  # 1 for horizontal flip
        frame = process_frame(frame)
        # Convert to RGB and display in notebook
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        _, buffer = cv2.imencode('.jpg', frame_rgb)
        img = buffer.tobytes()
        display(Image(data=img))
        clear_output(wait=True)  # Clear only the image display
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    # Redisplay the enable button after webcam is stopped
    display(enable_button)

def enable_webcam(b):
    global is_running
    if not is_running:
        is_running = True
        threading.Thread(target=start_webcam).start()

# Create and display the enable button only
enable_button = widgets.Button(description="Enable Webcam")
enable_button.on_click(enable_webcam)

# Display the enable button
display(enable_button)

Button(description='Enable Webcam', style=ButtonStyle())